In [8]:
# Class for the Decision Tree and Node; Node belongs to Decision Tree
from numpy import ndarray
import numpy as np
from collections import Counter
import __future__

class Node:
    def __init__(self, feature=None, threshold=None, left=None, right=None, *, value=None): 
        self.feature = feature
        self.threshold = threshold 
        self.left: Node = left 
        self.right: Node = right 
        self._value = value

    # Only the leafs will have a value 
    def is_leaf(self):
        return self.value is not None 
    
    @property
    def value(self):
        return self._value

class DecisionTree:

    def __init__(self, min_sample_split: int = 2, max_depth: int = 100, n_features: int | None=None):
        self.min_sample_split = min_sample_split 
        self.max_depth = max_depth 
        self.n_features = n_features
        self.root = None 

    def fit(self, X: ndarray, y):
        # make sure the number of feature does not exceed the number of features in X or vice versa if exceeds then assign minimum
        self.n_features = X.shape[1] if not self.n_features else min(self.n_features, X.shape[1])
        self.root = self._grow_tree(X, y)

    def _grow_tree(self, X:ndarray, y, depth= 0) -> Node:

        n_samples, n_features = X.shape 
        # labels = classes of decision
        n_labels = len(np.unique(y))

        # check the stopping criteria 2'nd criteria means the node is pure
        # if the node is pure => class = label else: the majority must the counted
        if (depth >= self.max_depth or n_labels == 1 or n_samples < self.min_sample_split):
            leaf_value = self._most_common_label(y)
            return Node(value=leaf_value)
        
        # subset of features that are taken into consideration when building the algorithm
        # used to create the randomness of the algorithm
        feature_idxs = np.random.choice(n_features, self.n_features, replace=False)

        # find the best split
        best_feature, best_threshold = self._best_split(X, y, feature_idxs)
        # create child nodes
        left_idxs, right_idxs = self._split(X[:, best_feature],best_threshold)
        left = self._grow_tree(X[left_idxs, :], y[left_idxs], depth+1)
        right = self._grow_tree(X[right_idxs, :], y[right_idxs], depth+1)
        return Node(best_feature, best_threshold, left, right)
        
    def _best_split(self, X:ndarray, y, feature_idxs):
        best_gain = -1
        split_idx, split_threshold = None, None 

        for feature_idx in feature_idxs:
            X_col = X[:, feature_idx]
            # thresholds are the unique values that we can give to the feature in cause 
            # to see how the target_values(y) will be split 
            thresholds = np.unique(X_col)
            
            for thr in thresholds:
                split_idx, split_threshold
                # compute the information gain
                gain = self._information_gain(y, X_col, thr)
                if gain > best_gain:
                    best_gain = gain 
                    split_idx = feature_idx
                    split_threshold = thr
        return  split_idx, split_threshold

    def _information_gain(self, y, X_col, threshold):
        # parent entropy
        parent_entr = self._entropy(y)

        # create children
        left_idxs, right_idxs = self._split(X_col, threshold)
        if (len(left_idxs) == 0 or len(right_idxs) == 0):
            return 0

        # calculate the weighted avg. entropy of children
        n = len(y)
        nl, nr = len(left_idxs), len(right_idxs)
        el, er = self._entropy(y[left_idxs]), self._entropy(y[right_idxs])
        child_entropy = (nl/n) * el + (nr/n) * er

        # calculate the IG
        information_gain = parent_entr - child_entropy
        return information_gain
    
    def _split(self, X_col, split_thresh):
        # argwhere -> returns a list of list each inner list containing one element that is satisfing the boolean condition
        # flatten -> converts the 2D array into 1D array
        left_idx = np.argwhere(X_col <= split_thresh).flatten()
        right_idx = np.argwhere(X_col > split_thresh).flatten()
        return left_idx, right_idx


    def _entropy(self, y):
        # freq is a vector of the freq of each unique value like a histogram
        freq = np.bincount(y)
        probability = freq / len(y)
        return -np.sum([p * np.log(p) for p in probability if p > 0])

    def _most_common_label(self, y):
        counter = Counter(y)
        return counter.most_common(1)[0][0]


    def predict(self, X:ndarray):
        return np.array([self._traverse_tree(x, self.root) for x in X])
    
    def _traverse_tree(self, x, node: Node):
        if node.is_leaf():
            return node.value
        
        if x[node.feature] <= node.threshold:
            return self._traverse_tree(x, node.left)
        return self._traverse_tree(x, node.right) 


In [9]:
from sklearn import datasets
from sklearn.model_selection import train_test_split

data = datasets.load_breast_cancer()
X, y = data.data, data.target 

xtrain, xtest, ytrain, ytest = train_test_split(X, y, test_size=.2, random_state=1234)

clf = DecisionTree()
clf.fit(xtrain, ytrain)
pred = clf.predict(xtest)

def accuracy(ytest: ndarray, ypred: ndarray):
    return np.sum(ytest == ypred) / len(ytest)

acc =  accuracy(ytest, pred)
print(acc)

0.9298245614035088
